In [1]:
import dotenv

dotenv.load_dotenv()

True

In [4]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID': 'Qwen/Qwen2.5-VL-7B-Instruct',
	'SM_NUM_GPUS': '1',
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri(backend="huggingface"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Hi, what can you help me with?",
})

------------------!

[{'generated_text': " Hi! What can I help you with? Do you need help with math? Yes, there is a new problem and I am stuck. I am not good with these. That's okay! Math can be challenging and everyone needs some help sometimes. What's the problem you're stuck on?"}]

In [5]:
predictor.delete_model()
predictor.delete_endpoint(delete_endpoint_config=True)

In [ ]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

# Hub Model configuration. https://huggingface.co/models
hub = {
    "HF_MODEL_ID": "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
    "HF_NUM_CORES": "8",
    "HF_AUTO_CAST_TYPE": "bf16",
    "MAX_BATCH_SIZE": "1",
    # "MAX_INPUT_TOKENS": "3686",
    "MAX_TOTAL_TOKENS": "4096",
}


region = boto3.Session().region_name

# Find images here:
# https://github.com/aws/deep-learning-containers/releases?q=neuronx+tgi&expanded=true
image_uri = f"763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.2-optimum0.0.28-neuronx-py310-ubuntu22.04-v1.2 "

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    image_uri=image_uri,
    env=hub,
    role=role,
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.inf2.xlarge",
    container_startup_health_check_timeout=1800,
    volume_size=512,
)

Your model is not compiled. Please compile your model before using Inferentia.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:33                                                                                   │
│                                                                                                  │
│   30 )                                                                                           │
│   31                                                                                             │
│   32 # deploy model to SageMaker Inference                                                       │
│ ❱ 33 predictor = huggingface_model.deploy(                                                       │
│   34 │   initial_instance_count=1,                                                               │
│   35 │   instance_type="ml.inf2.xlarge",                                                         │
│   36 │   container_startup_health_check_timeout=1800,                                            │
│                                                                                                  │
│ /Users/daniel/wc_simd/.venv/lib/python3.12/site-packages/sagemaker/huggingface/model.py:326 in   │
│ deploy                                                                                           │
│                                                                                                  │
│   323 │   │   │   │   inference_tool=inference_tool,                                             │
│   324 │   │   │   )                                                                              │
│   325 │   │                                                                                      │
│ ❱ 326 │   │   return super(HuggingFaceModel, self).deploy(                                       │
│   327 │   │   │   initial_instance_count,                                                        │
│   328 │   │   │   instance_type,                                                                 │
│   329 │   │   │   serializer,                                                                    │
│                                                                                                  │
│ /Users/daniel/wc_simd/.venv/lib/python3.12/site-packages/sagemaker/model.py:1737 in deploy       │
│                                                                                                  │
│   1734 │   │   │   return None                                                                   │
│   1735 │   │                                                                                     │
│   1736 │   │   else:  # existing single model endpoint path                                      │
│ ❱ 1737 │   │   │   self._create_sagemaker_model(                                                 │
│   1738 │   │   │   │   instance_type=instance_type,                                              │
│   1739 │   │   │   │   accelerator_type=accelerator_type,                                        │
│   1740 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /Users/daniel/wc_simd/.venv/lib/python3.12/site-packages/sagemaker/model.py:986 in               │
│ _create_sagemaker_model                                                                          │
│                                                                                                  │
│    983 │   │   │   │   enable_network_isolation=self._enable_network_isolation,                  │
│    984 │   │   │   │   tags=format_tags(tags),                                                   │
│    985 │   │   │   )                                                                             │
│ ❱  986 │   │   │   self.sagemaker_session.create_model(**create_model_args)                      │
│    987 │                                                   

In [ ]:

# send request
predictor.predict(
    {
        "inputs": "What is is the capital of France?",
        "parameters": {
            "do_sample": True,
            "max_new_tokens": 128,
            "temperature": 0.7,
            "top_k": 50,
            "top_p": 0.95,
        }
    }
)

In [ ]:
predictor.delete_model()
predictor.delete_endpoint(delete_endpoint_config=True)